# Tip Analysis

- Assess the percentage of tips by trip.
- Investigate geographical variations in tipping and the relationship between tipping amounts and trip distances.
- Study variations in tipping by time of day, week, and notable periods (e.g., holidays).
- Explore the impact of different payment types on tipping behavior.


## Part 1. Local Development

In [1]:
from pyspark.sql import SparkSession

# Create or retrieve a Spark session
spark = SparkSession.builder.appName("Tip Analysis").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/25 13:53:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/25 13:53:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
TRAIN_PROCESSED = "../../data/processed/yellow_tripdata_2024-01.parquet"

df = spark.read.parquet(TRAIN_PROCESSED)

In [3]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



### Step 1. Assess the Percentage of Tips by Trip


In [4]:
from pyspark.sql.functions import col, expr

df = df.withColumn("tip_percentage", col("tip_amount") / (col("fare_amount") + col("extra") + col("mta_tax")) * 100)
tip_percentage_by_trip = df.select("tip_percentage")
tip_percentage_by_trip.show()


+------------------+
|    tip_percentage|
+------------------+
|               0.0|
|26.785714285714285|
|10.989010989010989|
|14.285714285714285|
| 26.89075630252101|
|20.535714285714285|
|19.157088122605362|
|               0.0|
|               0.0|
|               0.0|
|               0.0|
|20.899470899470902|
|22.460317460317462|
|12.755102040816327|
|               0.0|
|               0.0|
|26.488095238095237|
|               0.0|
|27.777777777777775|
|52.083333333333336|
+------------------+
only showing top 20 rows



### 2. Investigate Geographical Variations in Tipping

In [6]:
from pyspark.sql.functions import col, avg

avg_tips_by_location = df.groupBy("PULocationID", "DOLocationID").agg(avg("tip_percentage").alias("avg_tip_percentage"))
avg_tips_by_location.show()


+------------+------------+------------------+
|PULocationID|DOLocationID|avg_tip_percentage|
+------------+------------+------------------+
|         236|         238|19.420470618085226|
|         148|         229|16.596049998818355|
|         107|         161|18.779140825018864|
|         229|         239|18.473992982875824|
|         231|         140|14.708558592369055|
|         163|         263|18.762452178520405|
|         148|         146|12.262024871267831|
|         163|           7|14.297723142802157|
|         151|         116|11.741421345294407|
|          75|          97|18.611820903910157|
|         114|         151|16.332967217040427|
|         231|          41|13.224851716469948|
|         232|          45|14.435281007242583|
|         116|         229|11.063125343574393|
|         132|         107|17.049076096067488|
|         264|         107|15.137765536989946|
|         229|          36|20.521484427531213|
|          43|           7| 17.61322964772887|
|          49

### Step 3. Study Variations in Tipping by Time of Day, Week, and Notable Periods


In [7]:
from pyspark.sql.functions import dayofweek, hour

df = df.withColumn("pickup_dayofweek", dayofweek("tpep_pickup_datetime"))
df = df.withColumn("pickup_hour", hour("tpep_pickup_datetime"))

# Average tip percentage by day of the week and hour
avg_tips_by_time = df.groupBy("pickup_dayofweek", "pickup_hour").agg(avg("tip_percentage").alias("avg_tip_percentage"))
avg_tips_by_time.show()


+----------------+-----------+------------------+
|pickup_dayofweek|pickup_hour|avg_tip_percentage|
+----------------+-----------+------------------+
|               3|         22|17.262825873235595|
|               3|         15| 17.63165910467149|
|               7|         21|17.695372036959583|
|               4|         10| 17.84951687890442|
|               5|         16|16.852673672017325|
|               6|         20|18.044354006106882|
|               1|          0|16.113172650781422|
|               6|          1| 35.80202971211582|
|               3|          1| 14.51643230823709|
|               7|          4|15.533714446235779|
|               2|          2|16.098495399747833|
|               6|         22|18.299501897516507|
|               3|         13|17.518601152940352|
|               2|         19| 17.91102047750753|
|               7|         14|18.193513357301786|
|               3|         18|20.090834696031795|
|               1|          7| 15.22467917334295|


### Step 4. Explore the Impact of Different Payment Types on Tipping Behavior


In [8]:
avg_tips_by_payment_type = df.groupBy("payment_type").agg(avg("tip_percentage").alias("avg_tip_percentage"))
avg_tips_by_payment_type.show()


+------------+--------------------+
|payment_type|  avg_tip_percentage|
+------------+--------------------+
|           1|  24.987643406796913|
|           3|0.007011956794025124|
|           2|-0.00497977152114...|
|           4|-0.13274535408945098|
|           0|   7.184890628619743|
+------------+--------------------+



### Save Results


In [9]:
# To save the results
ANALYSED_DATA_DIR = "../../results/analysed_data/"

avg_tips_by_location.write.format("parquet").save(f"{ANALYSED_DATA_DIR}avg_tips_by_location.parquet")
avg_tips_by_time.write.format("parquet").save(f"{ANALYSED_DATA_DIR}avg_tips_by_time.parquet")
avg_tips_by_payment_type.write.format("parquet").save(f"{ANALYSED_DATA_DIR}avg_tips_by_payment_type.parquet")

In [10]:
spark.stop()


## Part 2. Running on GCP

In [15]:
# Set bucket name
bucket_name = "spbd-nyc-taxi-bucket"
script_name = "tip_analysis.py"

# Upload the Python script
!gsutil cp ./{script_name} gs://{bucket_name}/scripts/

# Upload processed data
# !gsutil -m cp -r ../../data/processed/yellow_tripdata_2024-01.parquet gs://{bucket_name}/data/processed

Copying file://./tip_analysis.py [Content-Type=text/x-python]...
/ [1 files][  2.0 KiB/  2.0 KiB]                                                
Operation completed over 1 objects/2.0 KiB.                                      


In [12]:
cluster_name = "spbd-nyc-taxi-cluster"
region = "europe-west9"
machine_type="n2-standard-2"

!gcloud dataproc clusters create {cluster_name} \
    --region={region} \
    --zone={region}-a \
    --master-machine-type={machine_type} \
    --worker-machine-type={machine_type} \
    --num-workers=2 \
    --image-version=2.0-debian10 \
    --scopes=default


Waiting on operation [projects/epita-spbd-nyc-da/regions/europe-west9/operations/727c1c06-cc03-3eb5-9703-5fe614ee956d].
Waiting for cluster creation operation...                                      
Waiting for cluster creation operation...done.                                 
Created [https://dataproc.googleapis.com/v1/projects/epita-spbd-nyc-da/regions/europe-west9/clusters/spbd-nyc-taxi-cluster] Cluster placed in zone [europe-west9-a].


In [16]:
!gcloud dataproc jobs submit pyspark \
    gs://spbd-nyc-taxi-bucket/scripts/{script_name} \
    --cluster={cluster_name} \
    --region={region}

Job [dfeff9ec976a44f5954c582919e2b9e6] submitted.
Waiting for job output...
24/05/25 12:11:23 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
24/05/25 12:11:23 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
24/05/25 12:11:23 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
24/05/25 12:11:23 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
24/05/25 12:11:23 INFO org.sparkproject.jetty.util.log: Logging initialized @3305ms to org.sparkproject.jetty.util.log.Slf4jLog
24/05/25 12:11:23 INFO org.sparkproject.jetty.server.Server: jetty-9.4.40.v20210413; built: 2021-04-13T20:42:42.668Z; git: b881a572662e1943a14ae12e7e1207989f218b74; jvm 1.8.0_412-b08
24/05/25 12:11:23 INFO org.sparkproject.jetty.server.Server: Started @3416ms
24/05/25 12:11:23 INFO org.sparkproject.jetty.server.AbstractConnector: Started ServerConnector@3ec3186f{HTTP/1.1, (http/1.1)}{0.0.0.0:42959}
24/05/25 12:11:24 INFO org.apache.hadoop.yarn.client.RMPro

In [17]:
!gcloud dataproc jobs list --cluster=spbd-nyc-taxi-cluster --region=europe-west9

JOB_ID                            TYPE     STATUS
dfeff9ec976a44f5954c582919e2b9e6  pyspark  DONE
d255986775b54cf78ad6d968241cac4f  pyspark  ERROR


In [18]:
!gcloud dataproc clusters delete spbd-nyc-taxi-cluster --region=europe-west9 --quiet

Waiting on operation [projects/epita-spbd-nyc-da/regions/europe-west9/operations/4d72374c-2092-31a6-8f8a-65a6facadd42].
Waiting for cluster deletion operation...done.                                 
Deleted [https://dataproc.googleapis.com/v1/projects/epita-spbd-nyc-da/regions/europe-west9/clusters/spbd-nyc-taxi-cluster].
